In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
# 导入评估指标模块
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
# 导入表格库
import prettytable
# 导入dot插件库
import pydotplus
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [44]:
labelled_data = pd.read_csv('labelled_whitelist_shop2.csv')
labelled_data.shape
# labelled_data['kmeans_label']

(536, 17)

In [45]:
labelled_data.info()
# labelled_data.tail(20)
print(labelled_data['kmeans.labels'].value_counts())
print(labelled_data['agglomerative.average.labels'].value_counts())
print(labelled_data['agglomerative.complete.labels'].value_counts())
print(labelled_data['meanshift.labels'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   manage_shop_indicator          536 non-null    float64
 1   officialstore_indicator        536 non-null    float64
 2   preferred_shop_indicator       536 non-null    float64
 3   crossborder_indicator          536 non-null    float64
 4   shop_category                  536 non-null    float64
 5   new_seller_flag                536 non-null    float64
 6   seller_centre_login_L30D       536 non-null    float64
 7   shop_sku_number                536 non-null    float64
 8   shop_follower_number           536 non-null    float64
 9   shop_L180D_order               536 non-null    float64
 10  weighted_shop_rating           536 non-null    float64
 11  kmeans.labels                  536 non-null    int64  
 12  agglomerative.average.labels   536 non-null    int

In [109]:
#process non-whitelist data
non_whitelist = pd.read_csv('non_whitelist_filled.csv')
nw_data = non_whitelist.groupby("shop_index").mean()
nw_data = nw_data.drop(columns=['decorated_indicator','Unnamed: 0','performance_date','masked_item_impression','masked_order','masked_shop_page_view','masked_shop_click_from_search','masked_campaign_tab_click','masked_other_tab_click'])
nw_data.head(20)
nw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26018 entries, 119700 to 1486953315
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   manage_shop_indicator     26018 non-null  float64
 1   officialstore_indicator   26018 non-null  float64
 2   preferred_shop_indicator  26018 non-null  float64
 3   crossborder_indicator     26018 non-null  float64
 4   shop_category             26018 non-null  float64
 5   new_seller_flag           26018 non-null  float64
 6   seller_centre_login_L30D  26018 non-null  float64
 7   shop_sku_number           26018 non-null  float64
 8   shop_follower_number      26018 non-null  float64
 9   shop_L180D_order          26018 non-null  float64
 10  weighted_shop_rating      26018 non-null  float64
dtypes: float64(11)
memory usage: 2.4 MB


In [110]:
# agglomerative.average.labels 
#define x,y
x = labelled_data.iloc[:,:11]
y = labelled_data.iloc[:,-5]
#get train/test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2018)


In [116]:
#Decision Tree Classifier
#model training
dt_model = DecisionTreeClassifier(random_state=2018)
dt_model.fit(x_train, y_train)

#test the model with test dataset
pre_y = dt_model.predict(x_test)
accuracy_s = accuracy_score(y_test, pre_y)
print("DTC classifier: ",accuracy_s)
y_score = dt_model.predict_proba(x_test)

y_score

#prediction on non-whitelist data
prediction1 = dt_model.predict(nw_data)
prediction1
prob1 = dt_model.predict_proba(nw_data)
prob1[0:20]

DTC classifier:  0.8757763975155279


array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [117]:
#test accuracy
accuracy_s

0.8757763975155279

In [118]:
# training accuracy
pre_y_train = dt_model.predict(x_train)
accuracy_a = accuracy_score(y_train, pre_y_train)
accuracy_a

1.0

In [143]:
#random forest(currently good)
from sklearn.ensemble import RandomForestClassifier
# Create the model with 100 trees
rf_model = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')

# Fit on training data
rf_model.fit(x_train, y_train)

# Actual class predictions
rf_predictions = rf_model.predict(x_test)

# Probabilities for each class
rf_probs =rf_model.predict_proba(x_test)[:, 1]
# rf_probs
rf_predictions

#prediction on non_whitelist data
predictions = rf_model.predict(nw_data)
print(predictions[20:40])
proba = rf_model.predict_proba(nw_data)
print(proba[20:40])
proba = list(proba)
count = 0
# for i in proba:
#     if max(i)<0.7:
#         count += 1
print("unmatched num: ",count)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[[0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.39 0.34 0.09 0.17 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.43 0.31 0.09 0.16 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.51 0.32 0.   0.16 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.51 0.32 0.   0.16 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]
 [0.44 0.31 0.09 0.15 0.01]]
unmatched num:  0


In [144]:
# training accuracy 
ta_predictions = rf_model.predict(x_train)
train2 = accuracy_score(y_train,ta_predictions)
print(train2)

# test accuracy 
test2=accuracy_score(y_test,rf_predictions)
print(test2)


1.0
0.8385093167701864


In [121]:
#GradientBoostingClassifier
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve,auc
from sklearn import metrics
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#data
x = labelled_data.iloc[:,:11]
y = labelled_data.iloc[:,-5]
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=0,train_size=0.7)
ss_x = StandardScaler()
ss_y = StandardScaler()
x_train = ss_x.fit_transform(x_train)
x_test = ss_x.transform(x_test)

#model training
model_GBDT = GradientBoostingClassifier(random_state=10)
model_GBDT.fit(x_train,y_train)
y_pred = model_GBDT.predict(x_train)
y_predprob = model_GBDT.predict_proba(x_train)[:,1]

#predict on non whitelist data
model_GBDT.predict(nw_data.head(20))
prob_GBDT = model_GBDT.predict_proba(nw_data)
prob_GBDT
count_GBDT = 0
for i in range(len(prob_GBDT)):
    if np.max(prob_GBDT[i])<0.8:
        count_GBDT += 1
print(count_GBDT)
#     print(np.max(prob_GBDT[i]))

0


In [122]:
# training accuracy 
train3 = accuracy_score(y_train,y_pred)
print(train3)

# testing accuracy 
y_pred3= model_GBDT.predict(x_test)
test3 = accuracy_score(y_test,y_pred3)
print(test3)



1.0
0.8633540372670807


In [54]:
prob_GBDT

array([[7.88124443e-01, 1.99317783e-01, 1.45378123e-03, 1.04200096e-02,
        6.83983100e-04],
       [7.91038688e-01, 2.00054799e-01, 1.45915687e-03, 6.76084330e-03,
        6.86512261e-04],
       [7.88124443e-01, 1.99317783e-01, 1.45378123e-03, 1.04200096e-02,
        6.83983100e-04],
       ...,
       [7.88124443e-01, 1.99317783e-01, 1.45378123e-03, 1.04200096e-02,
        6.83983100e-04],
       [7.88124443e-01, 1.99317783e-01, 1.45378123e-03, 1.04200096e-02,
        6.83983100e-04],
       [7.96500121e-01, 1.94531875e-01, 1.46923108e-03, 6.80752103e-03,
        6.91252030e-04]])

In [9]:
#XGBoost (----------------to be edited, 还是报错)
import numpy as np
import pandas as pd 
import xgboost as xgb
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

In [10]:
x = labelled_data.iloc[:,:-4]
y = labelled_data.iloc[:,-4]
X_train, X_test, y_train, y_test = train_test_split(x, y,random_state=0,train_size=0.7)

In [104]:
# params = {
#     'booster': 'gbtree',
#     'objective': 'multi:softmax',
#     'num_class': 3,
#     'gamma': 0.1,
#     'max_depth': 6,
#     'lambda': 2,
#     'subsample': 0.7,
#     'colsample_bytree': 0.75,
#     'min_child_weight': 3,
#     'eta': 0.1,
#     'seed': 1,
#     'nthread': 4,
# }

# plst = list(params.items())

# dtrain = xgb.DMatrix(X_train, y_train) # 生成数据集格式
# num_rounds = 500
# model = xgb.train(plst, dtrain, num_rounds) # xgboost模型训练

# # 对测试集进行预测
# dtest = xgb.DMatrix(X_test)
# y_pred = model.predict(dtest)

# # 计算准确率
# accuracy = accuracy_score(y_test,y_pred)
# print("accuarcy: %.2f%%" % (accuracy*100.0))

# # 显示重要特征
# plot_importance(model)
# plt.show()

In [12]:
# agglomerative.complete.labels 
#define x,y
x1 = labelled_data.iloc[:,:11]
y1 = labelled_data.iloc[:,-4]
#get train/test data
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.3, random_state=2018)





In [13]:
#Decision Tree Classifier
#model training
dt_model1 = DecisionTreeClassifier(random_state=2018)
dt_model1.fit(x1_train, y1_train)
#test the model with test dataset
pre_y1 = dt_model1.predict(x1_test)
accuracy_s1 = accuracy_score(y1_test, pre_y1)
accuracy_s1
y1_score = dt_model1.predict_proba(x1_test)
y1_score

#prediction on non-whitelist data
prediction1 = dt_model1.predict(nw_data)
prediction1
prob1 = dt_model1.predict_proba(nw_data)
prob1[0:20]

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [14]:
#random forest(currently good)
from sklearn.ensemble import RandomForestClassifier
# Create the model with 100 trees
rf_model1 = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
rf_model1.fit(x1_train, y1_train)

# Actual class predictions
rf_predictions1 = rf_model1.predict(x1_test)

# Probabilities for each class
rf_probs1 =rf_model1.predict_proba(x1_test)[:, 1]
# rf_probs
rf_predictions

#prediction on non_whitelist data
predictions1 = rf_model1.predict(nw_data)
print(predictions1[20:40])
proba1 = rf_model1.predict_proba(nw_data)
print(proba1[20:40])
proba1 = list(proba1)
count1 = 0
for i in proba1:
    if max(i)<0.7:
        count1 += 1
print("unmatched num: ",count1)

[0 0 0 2 0 1 0 0 0 2 1 0 0 0 0 0 0 0 0 0]
[[0.92 0.06 0.01 0.   0.01]
 [0.84 0.1  0.06 0.   0.  ]
 [0.84 0.12 0.03 0.01 0.  ]
 [0.26 0.19 0.41 0.12 0.02]
 [0.87 0.07 0.04 0.   0.02]
 [0.37 0.57 0.03 0.   0.03]
 [0.79 0.12 0.09 0.   0.  ]
 [0.78 0.04 0.14 0.02 0.02]
 [0.81 0.12 0.06 0.   0.01]
 [0.2  0.02 0.74 0.04 0.  ]
 [0.28 0.7  0.   0.   0.02]
 [0.87 0.1  0.03 0.   0.  ]
 [0.84 0.12 0.03 0.01 0.  ]
 [0.9  0.09 0.01 0.   0.  ]
 [0.82 0.11 0.06 0.   0.01]
 [0.84 0.12 0.03 0.01 0.  ]
 [0.74 0.2  0.04 0.02 0.  ]
 [0.91 0.03 0.04 0.02 0.  ]
 [0.85 0.13 0.   0.01 0.01]
 [0.9  0.08 0.02 0.   0.  ]]
unmatched num:  9499


In [15]:
x1 = labelled_data.iloc[:,:11]
y1 = labelled_data.iloc[:,-4]
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=0,train_size=0.7)
ss_x = StandardScaler()
ss_y = StandardScaler()
x_train = ss_x.fit_transform(x_train)
x_test = ss_x.transform(x_test)

#model training
model_GBDT = GradientBoostingClassifier(random_state=10)
model_GBDT.fit(x_train,y_train)
y_pred = model_GBDT.predict(x_train)
y_predprob = model_GBDT.predict_proba(x_train)[:,1]

#predict on non whitelist data
model_GBDT.predict(nw_data.head(20))
prob_GBDT = model_GBDT.predict_proba(nw_data)
prob_GBDT
count_GBDT = 0
for i in range(len(prob_GBDT)):
    if np.max(prob_GBDT[i])>0.9:
        count_GBDT += 1
print(count_GBDT)
#     print(np.max(prob_GBDT[i]))

ValueError: Number of features of the model must match the input. Model n_features is 13 and input n_features is 11 